In [15]:
import sparknlp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml.feature import CountVectorizer

In [2]:
sparkSession = sparknlp.start()

In [3]:
df = sparkSession.createDataFrame([(1,'Hola compañero, ¿cómo estás?'),
                                   (2, 'Yo estoy bastante bien, gracias'),
                                   (3, 'Vamos a hacer este ejercicio de NLP')],
                                schema = StructType([StructField('id', IntegerType(), True),
                                                    StructField('text', StringType(), True)]))
df.show()                                   

+---+--------------------+
| id|                text|
+---+--------------------+
|  1|Hola compañero, ¿...|
|  2|Yo estoy bastante...|
|  3|Vamos a hacer est...|
+---+--------------------+



In [4]:
assemblerConfig = sparknlp.DocumentAssembler().setInputCol('text')\
                                            .setOutputCol('document')
dfAssembled = assemblerConfig.transform(df)
dfAssembled.show(truncate = False)

+---+-----------------------------------+-----------------------------------------------------------------------------+
|id |text                               |document                                                                     |
+---+-----------------------------------+-----------------------------------------------------------------------------+
|1  |Hola compañero, ¿cómo estás?       |[{document, 0, 27, Hola compañero, ¿cómo estás?, {sentence -> 0}, []}]       |
|2  |Yo estoy bastante bien, gracias    |[{document, 0, 30, Yo estoy bastante bien, gracias, {sentence -> 0}, []}]    |
|3  |Vamos a hacer este ejercicio de NLP|[{document, 0, 34, Vamos a hacer este ejercicio de NLP, {sentence -> 0}, []}]|
+---+-----------------------------------+-----------------------------------------------------------------------------+



In [5]:
tokenizerConfig = Tokenizer().setInputCols(['document'])\
                                    .setOutputCol('token')
tokenizerPipeline = Pipeline().setStages([tokenizerConfig])
dfTokenized = tokenizerPipeline.fit(dfAssembled).transform(dfAssembled)
dfTokenized.show(truncate = False)

+---+-----------------------------------+-----------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |text                               |document                                                                     |token                                                                                                                                                                                                                                                                                                           |
+---+-----------------------------------+-----------------------------------------------------------------------------+-------------------------------

In [6]:
wordsCleanerConfig = StopWordsCleaner().setInputCols(['token'])\
                                        .setOutputCol('token_cleaned')\
                                        .setCaseSensitive(False)\
                                        .setStopWords([',', 'yo', 'a', 'este', 'de'])
dfCleaned = wordsCleanerConfig.transform(dfTokenized)
dfCleaned.show(truncate = False)

+---+-----------------------------------+-----------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |text                               |document                                                                     |token                                                                                                                                                                                                                                     

In [7]:
finisherConfig = Finisher().setInputCols(['token_cleaned'])\
                            .setOutputCols(['final_token'])\
                            .setCleanAnnotations(True)
dfFinished = finisherConfig.transform(dfCleaned)
dfFinished.show(truncate = False)

+---+-----------------------------------+----------------------------------+
|id |text                               |final_token                       |
+---+-----------------------------------+----------------------------------+
|1  |Hola compañero, ¿cómo estás?       |[Hola, compañero, ¿cómo, estás, ?]|
|2  |Yo estoy bastante bien, gracias    |[estoy, bastante, bien, gracias]  |
|3  |Vamos a hacer este ejercicio de NLP|[Vamos, hacer, ejercicio, NLP]    |
+---+-----------------------------------+----------------------------------+



In [16]:
dfCountVector = dfFinished.select('id', 'final_token')
countVectorizerConfig = CountVectorizer().setInputCol('final_token')\
                                        .setOutputCol('features')
countVectorizedModel = countVectorizerConfig.fit(dfCountVector)
dfCountVectorized = countVectorizedModel.transform(dfCountVector)
dfCountVectorized.show(truncate = False)

+---+----------------------------------+---------------------------------------+
|id |final_token                       |features                               |
+---+----------------------------------+---------------------------------------+
|1  |[Hola, compañero, ¿cómo, estás, ?]|(13,[3,5,6,9,10],[1.0,1.0,1.0,1.0,1.0])|
|2  |[estoy, bastante, bien, gracias]  |(13,[2,4,7,11],[1.0,1.0,1.0,1.0])      |
|3  |[Vamos, hacer, ejercicio, NLP]    |(13,[0,1,8,12],[1.0,1.0,1.0,1.0])      |
+---+----------------------------------+---------------------------------------+



In [9]:
pipeLineComplete = Pipeline().setStages([assemblerConfig, tokenizerConfig, wordsCleanerConfig, finisherConfig])
modelComplete = pipeLineComplete.fit(df)
dfComplete = modelComplete.transform(df)
dfComplete.show(truncate = False)

+---+-----------------------------------+----------------------------------+
|id |text                               |final_token                       |
+---+-----------------------------------+----------------------------------+
|1  |Hola compañero, ¿cómo estás?       |[Hola, compañero, ¿cómo, estás, ?]|
|2  |Yo estoy bastante bien, gracias    |[estoy, bastante, bien, gracias]  |
|3  |Vamos a hacer este ejercicio de NLP|[Vamos, hacer, ejercicio, NLP]    |
+---+-----------------------------------+----------------------------------+

